## COVID19 plots

This notebook just plots some covid19 data, provided by https://datahub.io/core/covid-19

In [ ]:
from datapackage import Package
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import numpy as np
import datetime
from datetime import date
import scipy.optimize
import math

font = {'family': 'Times', 'size': 22}
matplotlib.rc('font', **font)

In [ ]:
package = Package('https://datahub.io/core/covid-19/datapackage.json')

In [ ]:
for resource in package.resources:
    if resource.descriptor['datahub']['type'] == 'derived/csv':
        print(resource.descriptor['name'])

In [ ]:
# Get the time series data
for resource in package.resources:
    if resource.descriptor['datahub']['type'] == 'derived/csv':
        if resource.descriptor['name'] == 'time-series-19-covid-combined_csv':
            data = resource.read(keyed=True)
        if resource.descriptor['name'] == 'countries-aggregated_csv':
            data_agg = resource.read(keyed=True)

In [ ]:
# Extract data points for specific U.S. states
def get_data_for_state(data, state):
    state_data = {}
    for key in data[0]:
        state_data[key] = []
    for point in data: 
        for key in point:
            if point['Province/State'] == state:
                state_data[key].append(point[key])
    return state_data

def get_data_for_country(data_agg, country):
    country_data = {}
    for key in data_agg[0]:
        country_data[key] = []
    for point in data_agg:
        for key in point:
            if point['Country'] == country:
                country_data[key].append(point[key])
    return country_data

data_ca = get_data_for_state(data, 'California')
data_ny = get_data_for_state(data, 'New York')
data_pa = get_data_for_state(data, 'Pennsylvania')

data_usa = get_data_for_country(data_agg, 'US')
data_canada = get_data_for_country(data_agg, 'Canada')

In [ ]:
countries = []
states = []
for point in data:
    if point['Country/Region'] == 'US':
        states.append(point['Province/State'])
        print(point)
    countries.append(point['Country/Region'])
countries = np.sort(np.unique(countries))

In [ ]:
states

In [ ]:
# Use curve_fit for exponential Y=a*exp(b*t)
# Based on: https://elm.nsupdate.info/maths.html
# Model: N(t) = N0*exp((ln(2)/T2)*(t-d)) 
# N0 is cases at t0, t is in days, N(t) is number 
# at time t, T2 is doubling time in days.

def exfunc(t,a,b):
    return a*np.exp(b*t)

def fitexfunc(x,y):
    popt, pcov = scipy.optimize.curve_fit(lambda t,a,b: exfunc(t,a,b),  x,  y, p0=(4, 0.1))
    return popt

In [ ]:
def plot_time_series(data_to_plot, keys_to_plot, yrange=[1.0e0, 1.0e4], title_key='Province/State',plotfit='False'):
    colors = ['b', 'r', 'g']
    plt.clf()
    fig = plt.figure(figsize=(16,4))
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    for i,key in enumerate(keys_to_plot):
        plt.plot_date(data_to_plot['Date'], data_to_plot[key], "."+colors[i]+"-", linewidth=2, markersize=10, label=key)
        if plotfit:
            x2 = np.arange(len(data_to_plot[key]))
            popt = fitexfunc(x2,data_to_plot[key])
            fitlabel = 'fit: N0=%5.2f, ' %popt[0] + 'T2=%5.2f' %(math.log(2)/popt[1]) 
            plt.plot_date(data_to_plot['Date'], exfunc(x2,*popt), colors[i]+"--", linewidth=2, markersize=10, label=fitlabel)
    plt.xticks(rotation=90)
    plt.yscale('log')
    plt.xlim(datetime.date(2020, 3, 9), date.today())
    plt.ylim(yrange[0],yrange[-1])
    plt.xlabel('Date')
    plt.legend(loc='best')
    plt.title(data_to_plot[title_key][0])
    ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=100))
    plt.grid()
    plt.show()

In [ ]:
plot_ranges = [1.0e0, 1.0e6]
plot_keys = ['Confirmed', 'Deaths']

In [ ]:
plot_time_series(data_ca, plot_keys, plot_ranges)

In [ ]:
plot_time_series(data_ny, plot_keys, plot_ranges)

In [ ]:
plot_time_series(data_pa, plot_keys, plot_ranges)

In [ ]:
plot_time_series(data_usa, plot_keys, plot_ranges, title_key='Country',plotfit='True')

In [ ]:
plot_time_series(data_canada, plot_keys, plot_ranges, title_key='Country')